# Naive Bayes Classifier Model

The code here represents the Naive Bayes Classifier machine learning model to predict whether a person is affected by covid or pneumonia or whether the person is in normal state using the chest X-Ray. The Chest X-Ray images have been pre-proccessed and then the features have been extracted and stored in .mat files. We now load these files and create the Naive Bayes Classifier model.

In [1]:
# Importing the required packages
from utils import *

import os
import pywt
import cv2
import glob
import itertools
import numpy as np
import seaborn as sn
import pandas as pd
import scipy.io as sio
import skimage.io as io
import tensorflow as tf
import matplotlib.pyplot as plt

from sklearn import metrics
from sklearn.svm import SVC
from tensorflow import keras
from skimage import exposure
from scipy.stats import skew
from scipy.stats import kurtosis
from sklearn import preprocessing
from IPython.display import Image
from skimage.color import rgb2gray
from keras.utils import to_categorical
from skimage.feature import greycomatrix
from keras.callbacks import EarlyStopping
from sklearn.metrics import roc_curve, auc
from sklearn.decomposition import KernelPCA
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import LabelEncoder
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.neural_network import MLPClassifier
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from skimage.transform import  rescale,resize
from sklearn.preprocessing import MinMaxScaler
from skimage.util import img_as_uint,img_as_ubyte
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [2]:
# Mentioning the working directory
source_dir='./'

First we are taking the matlab file, loading the data.

In [3]:
# Accessing covid.mat file and getting the data from the file
covid_features=sio.loadmat(os.path.join(source_dir,'covid.mat')) 
covid_features=covid_features['covid'] 
# Accessing normal.mat file and getting the data from the file
normal_features=sio.loadmat(os.path.join(source_dir,'normal.mat')) 
normal_features=normal_features['normal']  
# Accessing pneumonia.mat file and getting the data from the file
pneumonia_features=sio.loadmat(os.path.join(source_dir,'pneumonia.mat')) 
pneumonia_features=pneumonia_features['pneumonia']  

The scores are stored in X. Then Y is taken as target label.

In [4]:
# Extracting the scores-i.e the inputs and storing it in X
X=np.concatenate((covid_features[:,:-1],normal_features[:,:-1],pneumonia_features[:,:-1]), axis=0)
# Extracting the target labels, the last column alone
y=np.concatenate((covid_features[:,-1],normal_features[:,-1],pneumonia_features[:,-1]), axis=0)

Now we are doing normalising the data.


In [5]:
# Normalization of the data between 0 and 1
min_max_scaler=MinMaxScaler()
X = min_max_scaler.fit_transform(X)

Then we are sinking it to 64 components with the help of kernel.


In [6]:
# We use Kernel to reduce the feature components to 64 for the input data
transformer = KernelPCA(n_components=64, kernel='linear')
X = transformer.fit_transform(X)

Here we are doing train, test,split....20% in the original dataset and 25% training dataset that we are splitting.


In [7]:
# We do the splitting of data set and set the training data to be 80% ,i.e, test data = 20%
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)
# From that 80%, test data fraction is set as 25% to get a better output as there is more randomness in the dataset for testing
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.25, random_state=1)

Printing the shape of train data,train label,test data and test label.


In [8]:
# Printing the size for each of the data
print("Size of train data:", np.shape(X_train))
print("Size of train label:", np.shape(y_train))
print("Size of test data:", np.shape(X_test))
print("Size of test label:", np.shape(y_test))

Size of train data: (225, 64)
Size of train label: (225,)
Size of test data: (76, 64)
Size of test label: (76,)


Now what we are creating is Gaussian Naive Bayes model.

In [9]:
# Creating the Gaussian Naive Bayes Classification object
clf = GaussianNB()
# Training the model
clf.fit(X_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

Now we are predicting the label (that is we are testing the model)...so we are sending the testing data X_test.

In [10]:
# Predicting/ testing the model
y_pred = clf.predict(X_test)
#print(y_pred)
#Obtaining the evaluation metrics for the model
conf_mat = confusion_matrix(y_test, y_pred) # Confusion matrix
accuracy = accuracy_score(y_test, y_pred) # Accuracy of prediction
precision = precision_score(y_test, y_pred, average='macro') # Precision
recall = recall_score(y_test, y_pred, average='macro') # Recall
f1 = f1_score(y_test, y_pred, average='macro') # F1-score

Then we are computing the following and printing.

In [11]:
print("The Accuracy of the model is: " + str(accuracy))

The Accuracy of the model is: 0.7631578947368421


In [15]:
print("The Confusion matrix is: ")
conf_mat

The Confusion matrix is: 


array([[18,  9,  3],
       [ 0, 14,  2],
       [ 1,  3, 26]], dtype=int64)

In [12]:
print("The Precision of the model is: " + str(precision))

The Precision of the model is: 0.7748465456445083


In [13]:
print("The Recall of the model is: " + str(recall))

The Recall of the model is: 0.7805555555555556


In [14]:
print("The F1-Score for the model is: " + str(f1))

The F1-Score for the model is: 0.7512731868703767
